In [ ]:
import pandas as pd
import requests
from census import Census
from us import states
from tabulate import tabulate 

In [ ]:
api_key = open('census_api_key.txt').read().strip()
c=Census(api_key)

In [ ]:
# Full API variable list available here https://api.census.gov/data/2016/acs/acs5/variables/
categories = [
     'NAME', # county name
     'B01001_001E', # Total population
     'B15002_001E', # Total population 25 and over
     'B19013_001E', # Median income
     'B25077_001E', # Median home value
     'B15011_001E', # Total population age 25+ years with a bachelor's degree or higher
     'B03002_003E', # Not Hispanic or Latino!!White alone
     'B03002_004E', # Not Hispanic or Latino!!Black or African American alone
     'B02001_004E', # American Indian and Alaska Native Alone
     'B03002_006E', # Not Hispanic or Latino!!Asian alone
     'B03002_007E', # Not Hispanic or Latino!!Native Hawaiian and Other Pacific Islander alone
     'B03002_008E', # Not Hispanic or Latino!!Some other race alone
     'B03002_009E', # Not Hispanic or Latino!!Two or more races
     'B03002_012E', # Hispanic or Latino
]

In [ ]:
#Function to run API scraper:

def get_acs_data(state_code, county_code, timeperiod, metro_area):
    results = c.acs5.state_county_tract(
        categories,
        state_code,
        county_code, 
        Census.ALL,
        year = timeperiod
    )

    return [ {
        'geoid': res['state'] + res['county'] + res['tract'],
        'name': res['NAME'],
        'total_population': res['B01001_001E'],
        'total_population_25_over': res['B15002_001E'],
        'median_income': res['B19013_001E'],
        'median_home_value': res['B25077_001E'],
        'educational_attainment': res['B15011_001E'],
        'white_alone': res['B03002_003E'],
        'black_alone': res['B03002_004E'],
        'native_alone': res['B02001_004E'],
        'asian_alone': res['B03002_006E'],
        'native_hawaiian_pacific_islander': res['B03002_007E'],
        'some_other_race_alone': res['B03002_008E'],
        'two_or_more': res['B03002_009E'],
        'hispanic_or_latino': res['B03002_012E'],
        'city': city,
        'metro_area': metro_area
    } for res in results ]

In [ ]:
metro_area_counties = pd.read_csv(
    '../SA_eastside_demographic/metrolist2.csv',
    dtype = {
        "metro_area_code": str,
        "county_code": str
    }
).rename(
        columns={
            "county_code": "full_geo_code",
            "county_name":  "county_name",
            "metro_area_name":"metro_area"

        }
    )

metro_area_counties.head()

In [ ]:
del metro_area_counties ['Unnamed: 4']

In [ ]:
metro_area_counties.columns

In [ ]:
metro_area_counties["state_code"]= metro_area_counties["full_geo_code"].apply(lambda x: str(x)[0:2])
metro_area_counties["county_code"] =  metro_area_counties["full_geo_code"].apply(lambda x: str(x)[2:5])
metro_area_counties.head()

In [ ]:
census_data = []
for index, county in metro_area_counties.iterrows():
    print(county['county_name'])
    
    census_data += get_acs_data(
    county['state_code'], 
    county['county_code'], 
    2010, 
    county['metro_area']
#    county['city']
    )

census_data = pd.DataFrame(census_data)

In [ ]:
census_data.head()